In [56]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sys, os,git
from pathlib import Path
from pprint import pprint
import subprocess
from ruamel.yaml import YAML
from ruamel.yaml.comments import CommentedMap
yaml = YAML()
yaml.preserve_quotes = True

os.environ['PATH'] = '/g/data/vk83/apps/payu/1.1.3/bin:' + os.environ['PATH']
print("PATH:", os.environ['PATH'])
!python -c 'import payu; print(payu)'

try:
    #import yaml
    import f90nml  # from https://f90nml.readthedocs.io/en/latest/
except ImportError:  # BUG: don't get this exception if payu module loaded, even if on python 2.6.6
    print('\nFatal error: modules not available.')
    print('On NCI, do the following and try again:')
    print('   module use /g/data/hh5/public/modules; module load conda/analysis3\n')
    raise

PATH: /g/data/vk83/apps/payu/1.1.3/bin:/g/data/hh5/public/apps/miniconda3/envs/analysis3-unstable/bin:/g/data/hh5/public/apps/miniconda3/condabin:/opt/pbs/default/bin:/opt/nci/bin:/opt/bin:/opt/Modules/v4.3.0/bin:/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/local/pbs/bin
<module 'payu' from '/g/data/vk83/apps/payu/1.1.3/lib/python3.9/site-packages/payu/__init__.py'>


In [3]:
# # ======================================================
# # from https://gist.github.com/paulkernstock/6df1c7ad37fd71b1da3cb05e70b9f522
# from yaml.representer import SafeRepresenter


# class LiteralString(str):
#     pass


# def change_style(style, representer):
#     def new_representer(dumper, data):
#         scalar = representer(dumper, data)
#         scalar.style = style
#         return scalar
#     return new_representer

# represent_literal_str = change_style('|', SafeRepresenter.represent_str)
# yaml.add_representer(LiteralString, represent_literal_str)
# # ======================================================

In [3]:
dir_ensemble = '/g/data/tm70/ml0072/COMMON/ensemble'
sys.path.append(dir_ensemble)

dir_utils = '/g/data/tm70/ml0072/COMMON/om3-utils/om3utils'
sys.path.append(dir_utils)

In [4]:
yaml_text = 'ensemble.yaml'
yamlfile = Path(dir_ensemble,yaml_text)
with open(yamlfile,'r') as f:
        indata = yaml.load(f)
# indata = yaml.load(open(yamlfile, 'r'), Loader=yaml.SafeLoader)
indata

{'template': 'test/1deg_jra55_ryf', 'startfrom': 'rest', 'nruns': 0, 'namelists': {'MOM_input.nml': {'mom6_input_nml': {'DT': [1800.0, 3600.0, 7200.0]}}}}

In [5]:
template = indata['template']
print(f'template: {template}')
templatepath = Path(dir_ensemble,template)
print(f'templatepath: {templatepath}')
templaterepo = git.Repo(templatepath)
print(templaterepo)
print(f'current branch: {templaterepo.active_branch.name}')

template: test/1deg_jra55_ryf
templatepath: /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf
<git.repo.base.Repo '/g/data/tm70/ml0072/COMMON/ensemble/.git/modules/test/1deg_jra55_ryf'>
current branch: master


In [49]:
import yaml
with open('/g/data/tm70/ml0072/COMMON/ensemble/ensemble_mom.yaml','r') as f_yaml:
    data_yaml = yaml.safe_load(f_yaml)

var_names = list(data_yaml['MOM_list'].keys())
new_values = [values[0] if isinstance(values,list) else value for values in data_yaml['MOM_list'].values()]
new_values

[1800.0, 1800.0]

In [6]:
import re
with open('/g/data/tm70/ml0072/MOM6-CICE6/PROFILING/3_2using_scripts/clx-run-o1344i96c96d48-fcpl-s-fm-F-fr-c_no_diags/MOM_input','r') as f:
    lines = f.readlines()


In [53]:
import re
import yaml

class MOM6InputParser:
    """
    A class to handle mom6 input files, allowing for reading, parsing, updating, and writing MOM_input
    """
    def __init__(self):
        self.header_pattern = re.compile(r"^! === (.+) ===")
        self.block_list = ['KPP%', '%KPP', 'CVMix_CONVECTION%', '%CVMix_CONVECTION', 'CVMIX_DDIFF%', '%CVMIX_DDIFF']
        self.block_pattern = re.compile(r"|".join(re.escape(item) for item in self.block_list))
        self.param_list = {}
        self.commt_list = {}
        self.current_header = None
        self.current_var = None
        self.current_value = []
        self.current_comment = []
        
    def read_lines(self,path_MOM_input):
        """
        Read an existing MOM_input
        """
        with open(path_MOM_input,'r') as f:
            self.lines = f.readlines()
            
    def parse_lines(self, lines):
        """
        Parse lines from the MOM_input
        """
        for line in lines:
            header_match = self.header_pattern.match(line)
            if header_match:
                self._save_current_var()
                self._start_new_header(header_match.group(1).strip())
            elif '=' in line:
                self._save_current_var()
                self._parse_param_value(line)
            elif self.block_pattern.search(line):
                self._save_current_var()
                self._start_new_block(line.strip())
            elif line.strip().startswith('!'):
                if self.current_var:
                    self.current_comment.append(line.strip())
        self._save_current_var()

    def _save_current_var(self):
        if self.current_var:
            var_name = self.current_var
            value = ''.join(self.current_value).strip()
            comment = '\n'.join(self.current_comment).strip()
            self._save_var_to_param_list(var_name, value, comment)

    def _save_var_to_param_list(self, var_name, value, comment):
        if self.current_header:
            self._initialize_header()
            self.param_list[self.current_header][var_name] = value
            self.commt_list[self.current_header][var_name] = comment

    def _initialize_header(self):
        if self.current_header not in self.param_list:
            self.param_list[self.current_header] = {}
            self.commt_list[self.current_header] = {}
            
    def _start_new_header(self, header):
        self.current_header = header
        self.current_var = None
        self.current_value = []
        self.current_comment = []
        self._initialize_header()

    def _parse_param_value(self, line):
        param, value = map(str.strip, line.split('=', 1))
        value_part = value.split('!')[0].strip()
        comment_part = value.split('!')[1].strip() if '!' in value else ''
        self.current_var = param
        self.current_value = value_part
        self.current_comment = [comment_part]

    def _start_new_block(self, block):
        self.current_var = block
        self.current_value = ''
        self.current_comment = ['']
        

    def update_MOM_input_from_list(self, var_names, new_values):
        """
        Update MOM_input parameters from a list of parameter names and associated values
        """
        if len(var_names) != len(new_values):
            raise ValueError('Lengths of var_names and new_values must be the same!')
        for var_name, new_value in zip(var_names, new_values):
            updated = False
            for header, variables in self.param_list.items():
                if var_name in variables:
                    self.param_list[header][var_name] = new_value
                    self.commt_list[header][var_name] = self.commt_list[header].get(var_name, '')
                    updated = True
                    break
            if not updated:
                print(f"parameter {var_name} not found!")

    def update_MOM_input_from_yaml(self, yaml_file):
        with open(yaml_file, 'r') as file:
            data = yaml.safe_load(file)
        if 'MOM_list' in data:
            yaml_params = data['MOM_list']
            all_var_names, all_new_values = zip(*((var_name, value) if not isinstance(value, list) 
                                                  else [(var_name, v) for v in value] 
                                                  for var_name, value in yaml_params.items()))
            self.update_MOM_input_from_list(all_var_names, all_new_values)

    def writefile_MOM_input(self, path_MOM_input, total_width=32):
        """
        Write the updated MOM_input to file
        """
        with open(path_MOM_input, 'w') as f:
            f.write("! This file was written by the script \n")
            f.write("! and records the non-default parameters used at run-time.\n")
            f.write("\n")
            for header, variables in self.param_list.items():
                f.write(f"! === {header} ===\n")
                for var, value in variables.items():
                    comment = self.commt_list[header].get(var, '')
                    if comment:
                        comment_lines = comment.split('\n')
                        param_str = f"{var} = {value}"
                        f.write(f"{param_str:<{total_width}} ! {comment_lines[0].strip()}\n")
                        for comment_line in comment_lines[1:]:
                            f.write(f"{'':<{total_width}} {comment_line.strip()}\n")
                    elif var in self.block_list:
                        f.write(f"{var}\n")
                    else:
                        f.write(f"{var} = {value}\n")
                f.write("\n")

# Initialize parser
parser = MOM6InputParser()

# Read lines from the file
path_MOM_input = '/g/data/tm70/ml0072/MOM6-CICE6/PROFILING/3_2using_scripts/clx-run-o1344i96c96d48-fcpl-s-fm-F-fr-c_no_diags/MOM_input'
parser.read_lines(path_MOM_input)

# Parse the lines
parser.parse_lines(parser.lines)

# # Update parameters from a YAML file
# yaml_file = 'params.yaml'
# parser.update_MOM_input_from_yaml(yaml_file)

# Write updated parameters to a file
output_path = '/g/data/tm70/ml0072/MOM6-CICE6/PROFILING/3_2using_scripts/clx-run-o1344i96c96d48-fcpl-s-fm-F-fr-c_no_diags/MOM_input_10'
parser.writefile_MOM_input(output_path)

In [ ]:
import re
import yaml
class mom6_input_Parser(object):
    def __init__(self):
        self.header_pattern = re.compile(r"^! === (.+) ===")
        self.block_list = ['KPP%', '%KPP', 'CVMix_CONVECTION%', '%CVMix_CONVECTION', 'CVMIX_DDIFF%', '%CVMIX_DDIFF']
        self.block_pattern = re.compile(r"KPP%|%KPP|CVMix_CONVECTION%|%CVMix_CONVECTION|CVMIX_DDIFF%|%CVMIX_DDIFF")
        self.param_list = {}
        self.commt_list = {}
        
        self.current_header = None
        self.current_var = None
        self.current_value = []
        self.current_comment = []
            
    def parse_lines(self, lines):
        for line in lines:
            header_match = self.header_pattern.match(line)
            if header_match:
                if self.current_var:
                    var_name = self.current_var
                    value = ''.join(self.current_value).strip()
                    comment = '\n'.join(self.current_comment).strip()
                    self._save_var_to_param_list(var_name, value, comment)

                self.current_header = header_match.group(1).strip()
                self.current_var = None
                self.current_value = []
                self.current_comment = []

                if self.current_header not in self.param_list:
                    self.param_list[self.current_header] = {}
                    self.commt_list[self.current_header] = {}
                continue

            if '=' in line:
                if self.current_var:
                    var_name = self.current_var
                    value = ''.join(self.current_value).strip()
                    comment = '\n'.join(self.current_comment).strip()
                    self._save_var_to_param_list(var_name, value, comment)

                param, value = line.split('=', 1)
                param = param.strip()
                tmp_value = value.split('!')[0].strip()
                tmp_commt = value.split('!')[1].strip() if '!' in value else ''

                self.current_var = param
                self.current_value = tmp_value
                self.current_comment = [tmp_commt]

            elif self.block_pattern.search(line):
                if self.current_var:
                    var_name = self.current_var
                    value = ''.join(self.current_value).strip()
                    comment = '\n'.join(self.current_comment).strip()
                    self._save_var_to_param_list(var_name, value, comment)

                self.current_var = line.strip()
                self.current_value = ''
                self.current_comment = ['']

            if line.strip().startswith('!'):
                if self.current_var:
                    self.current_comment.append(line.strip())
                continue

        if self.current_var:
            var_name = self.current_var
            value = ''.join(self.current_value).strip()
            comment = '\n'.join(self.current_comment).strip()
            self._save_var_to_param_list(var_name, value, comment)
            
    def _save_var_to_param_list(self, var_name, value, comment):
        if self.current_header:
            if self.current_header not in self.param_list:
                self.param_list[self.current_header] = {}
                self.commt_list[self.current_header] = {}
            self.param_list[self.current_header][var_name] = value
            self.commt_list[self.current_header][var_name] = comment


def writefile_MOM_input(path_MOM_input,param_list,commt_list,total_width = 32):
    with open(path_MOM_input,'w') as f:
        f.write("! This file was written by the script \n")
        f.write("! and records the non-default parameters used at run-time.\n")
        f.write("\n")
        for header,variables in param_list.items():
            f.write(f"! === {header} ===\n")
            for var,value in variables.items():
                comment = commt_list[header].get(var,'')
                if comment:
                    comment_lines = comment.split('\n')
                    param_str = f"{var} = {value}"
                    f.write(f"{param_str:<{total_width}} ! {comment_lines[0].strip()}\n")
                    for comment_line in comment_lines[1:]:
                        f.write(f"{'':<{total_width}} {comment_line.strip()}\n")
                elif var in block_list:
                    f.write(f"{var}\n")
                else:
                    f.write(f"{var} = {value}\n")
            f.write("\n")
            
parser = mom6_input_Parser()
parser.parse_lines(lines)
# change a param USE_CVMix_CONVECTION --> False
var_names = ['BBL_MIXING_AS_MAX','USE_LOTW_BBL_DIFFUSIVITY']
new_values = ['False','True']
#parser.update_MOM_input_from_list(var_names,new_values)
yaml_file = '/g/data/tm70/ml0072/COMMON/ensemble/ensemble_mom.yaml'
parser.update_MOM_input_from_yaml(yaml_file)

writefile_MOM_input('/g/data/tm70/ml0072/MOM6-CICE6/PROFILING/3_2using_scripts/clx-run-o1344i96c96d48-fcpl-s-fm-F-fr-c_no_diags/MOM_input_writefile7',
                    parser.param_list,
                    parser.commt_list,total_width = 32)


In [32]:
import re
import yaml
class mom6_input_Parser(object):
    def __init__(self):
        self.header_pattern = re.compile(r"^! === (.+) ===")
        self.block_list = ['KPP%', '%KPP', 'CVMix_CONVECTION%', '%CVMix_CONVECTION', 'CVMIX_DDIFF%', '%CVMIX_DDIFF']
        self.block_pattern = re.compile(r"KPP%|%KPP|CVMix_CONVECTION%|%CVMix_CONVECTION|CVMIX_DDIFF%|%CVMIX_DDIFF")

        self.param_list = {}
        self.commt_list = {}

        self.current_header = None
        self.current_var = None
        self.current_value = []
        self.current_comment = []
        
    def _save_var_to_param_list(self, var_name, value, comment):
        if self.current_header:
            if self.current_header not in self.param_list:
                self.param_list[self.current_header] = {}
                self.commt_list[self.current_header] = {}
            self.param_list[self.current_header][var_name] = value
            self.commt_list[self.current_header][var_name] = comment
            
    def update_MOM_input_from_list(self,var_names,new_values):
        if len(var_names)!=len(new_values):
            raise ValueError('Lengths of var_name and value must be the same!')
        for var_name, new_value in zip(var_names,new_values):
            updated = False
            for header, variables in self.param_list.items():
                if var_name in variables:
                    self.param_list[header][var_name] = new_value
                    self.commt_list[header][var_name] = self.commt_list[header].get(var_name,'')
                    updated = True
                    break
            if not updated:
                print('f"parameter {var_name} not found!"')

    def update_MOM_input_from_yaml(self,yaml_file):
        with open(yaml_file,'r') as f_yaml:
            data_yaml = yaml.safe_load(f_yaml)

        if 'MOM_list' in data_yaml:
            params_yaml = data_yaml['MOM_list']
            param_names = []
            new_values  = []
            for param, values in params_yaml.items():
                if isinstance(values,list):
                    param_names.extend([param]*len(values))
                    new_values.extend(values)
                else:
                    param_names.append(param)
                    new_values.extend(values)
            self.update_MOM_input_from_list(param_names,new_values)
                
            
    
    def parse_lines(self, lines):
        for line in lines:
            header_match = self.header_pattern.match(line)
            if header_match:
                if self.current_var:
                    var_name = self.current_var
                    value = ''.join(self.current_value).strip()
                    comment = '\n'.join(self.current_comment).strip()
                    self._save_var_to_param_list(var_name, value, comment)

                self.current_header = header_match.group(1).strip()
                self.current_var = None
                self.current_value = []
                self.current_comment = []

                if self.current_header not in self.param_list:
                    self.param_list[self.current_header] = {}
                    self.commt_list[self.current_header] = {}
                continue

            if '=' in line:
                if self.current_var:
                    var_name = self.current_var
                    value = ''.join(self.current_value).strip()
                    comment = '\n'.join(self.current_comment).strip()
                    self._save_var_to_param_list(var_name, value, comment)

                param, value = line.split('=', 1)
                param = param.strip()
                tmp_value = value.split('!')[0].strip()
                tmp_commt = value.split('!')[1].strip() if '!' in value else ''

                self.current_var = param
                self.current_value = tmp_value
                self.current_comment = [tmp_commt]

            elif self.block_pattern.search(line):
                if self.current_var:
                    var_name = self.current_var
                    value = ''.join(self.current_value).strip()
                    comment = '\n'.join(self.current_comment).strip()
                    self._save_var_to_param_list(var_name, value, comment)

                self.current_var = line.strip()
                self.current_value = ''
                self.current_comment = ['']

            if line.strip().startswith('!'):
                if self.current_var:
                    self.current_comment.append(line.strip())
                continue

        if self.current_var:
            var_name = self.current_var
            value = ''.join(self.current_value).strip()
            comment = '\n'.join(self.current_comment).strip()
            self._save_var_to_param_list(var_name, value, comment)
        
def writefile_MOM_input(path_MOM_input,param_list,commt_list,total_width = 32):
    with open(path_MOM_input,'w') as f:
        f.write("! This file was written by the script \n")
        f.write("! and records the non-default parameters used at run-time.\n")
        f.write("\n")
        for header,variables in param_list.items():
            f.write(f"! === {header} ===\n")
            for var,value in variables.items():
                comment = commt_list[header].get(var,'')
                if comment:
                    comment_lines = comment.split('\n')
                    param_str = f"{var} = {value}"
                    f.write(f"{param_str:<{total_width}} ! {comment_lines[0].strip()}\n")
                    for comment_line in comment_lines[1:]:
                        f.write(f"{'':<{total_width}} {comment_line.strip()}\n")
                elif var in block_list:
                    f.write(f"{var}\n")
                else:
                    f.write(f"{var} = {value}\n")
            f.write("\n")
            
parser = mom6_input_Parser()
parser.parse_lines(lines)
# change a param USE_CVMix_CONVECTION --> False
var_names = ['BBL_MIXING_AS_MAX','USE_LOTW_BBL_DIFFUSIVITY']
new_values = ['False','True']
#parser.update_MOM_input_from_list(var_names,new_values)
yaml_file = '/g/data/tm70/ml0072/COMMON/ensemble/ensemble_mom.yaml'
parser.update_MOM_input_from_yaml(yaml_file)

writefile_MOM_input('/g/data/tm70/ml0072/MOM6-CICE6/PROFILING/3_2using_scripts/clx-run-o1344i96c96d48-fcpl-s-fm-F-fr-c_no_diags/MOM_input_writefile7',
                    parser.param_list,
                    parser.commt_list,total_width = 32)
# parser.param_list
# parser.commt_list
#parser.writefile_MOM_input( '/g/data/tm70/ml0072/MOM6-CICE6/PROFILING/3_2using_scripts/clx-run-o1344i96c96d48-fcpl-s-fm-F-fr-c_no_diags/MOM_input_writefile6')

In [13]:
test

In [54]:
header_pattern = re.compile(r"^! === (.+) ===")
block_list = ['KPP%','%KPP','CVMix_CONVECTION%','%CVMix_CONVECTION','CVMIX_DDIFF%','%CVMIX_DDIFF']
block_pattern  = re.compile(r"KPP%|%KPP|CVMix_CONVECTION%|%CVMix_CONVECTION|CVMIX_DDIFF%|%CVMIX_DDIFF")

param_list = {}
commt_list = {}

current_header  = None
current_var     = None
current_value   = []
current_comment = []

def _save_var_to_param_list(var_name,value,comment):
    if current_header:
        if current_header not in param_list:                # save parameters and associated comments under each header
            param_list[current_header] = {}
            commt_list[current_header] = {}
            
        param_list[current_header][var_name] = value
        commt_list[current_header][var_name] = comment

for line in lines:
    
    header_match = header_pattern.match(line)               # check if line is a header
    if header_match:
        if current_var:
            var_name = current_var
            value    = ''.join(current_value).strip()
            comment  = '\n'.join(current_comment).strip()
            _save_var_to_param_list(var_name,value,comment)
            
        current_header = header_match.group(1).strip()
        current_var = None                                  # under new header, needs to reset param
        current_value = []                                  # under new header, needs to reset value
        current_comment = []                                # under new header, needs to reset comment

        if current_header not in param_list:                # keep the header even if there is no parameter existed
            param_list[current_header] = {}
            commt_list[current_header] = {}
        continue

    # progress params and values
    if '=' in line:
        if current_var:
            var_name = current_var
            value    = ''.join(current_value).strip()
            comment  = '\n'.join(current_comment).strip()
            _save_var_to_param_list(var_name,value,comment)
        param,value = line.split('=',1)
        param = param.strip()
        # separate value and inline comment
        tmp_value = value.split('!')[0].strip()                         # value
        tmp_commt = value.split('!')[1].strip() if '!' in value else '' # comments
        
        current_var = param
        current_value = tmp_value
        current_comment = [tmp_commt]

    elif block_pattern.search(line):                        # block_pattern
        if current_var:
            var_name = current_var
            value    = ''.join(current_value).strip()
            comment  = '\n'.join(current_comment).strip()
            _save_var_to_param_list(var_name,value,comment)
        current_var = line.strip()
        current_value = ''
        current_comment = ['']
        
    # add comments to current param
    if line.strip().startswith('!'):
        if current_var:
            current_comment.append(line.strip())
        continue
        
# last var
if current_var:
    var_name = current_var
    value    = ''.join(current_value).strip()
    comment  = '\n'.join(current_comment).strip()
    _save_var_to_param_list(var_name,value,comment)


def writefile_MOM_input(path_MOM_input,param_list,commt_list,total_width = 32):
    with open(path_MOM_input,'w') as f:
        f.write("! This file was written by the script \n")
        f.write("! and records the non-default parameters used at run-time.\n")
        f.write("\n")
        for header,variables in param_list.items():
            f.write(f"! === {header} ===\n")
            for var,value in variables.items():
                comment = commt_list[header].get(var,'')
                if comment:
                    comment_lines = comment.split('\n')
                    param_str = f"{var} = {value}"
                    f.write(f"{param_str:<{total_width}} ! {comment_lines[0].strip()}\n")
                    for comment_line in comment_lines[1:]:
                        f.write(f"{'':<{total_width}} {comment_line.strip()}\n")
                elif var in block_list:
                    f.write(f"{var}\n")
                else:
                    f.write(f"{var} = {value}\n")
            f.write("\n")

path_MOM_input = '/g/data/tm70/ml0072/MOM6-CICE6/PROFILING/3_2using_scripts/clx-run-o1344i96c96d48-fcpl-s-fm-F-fr-c_no_diags/MOM_input_writefile5'
writefile_MOM_input(path_MOM_input,param_list,commt_list)

In [27]:
# test - can delete

header_pattern = re.compile(r"^! === (.+) ===")
block_list = ['KPP%','%KPP','CVMix_CONVECTION%','%CVMix_CONVECTION','CVMIX_DDIFF%','%CVMIX_DDIFF']
block_pattern  = re.compile(r"KPP%|%KPP|CVMix_CONVECTION%|%CVMix_CONVECTION|CVMIX_DDIFF%|%CVMIX_DDIFF")

param_list = {}
commt_list = {}

current_header  = None
current_var     = None
current_value   = []
current_comment = []

def _save_var_to_param_list(var_name,value,comment):
    # Each parameter is included under a header
    if current_header not in param_list:
        param_list[current_header] = {}
        commt_list[current_header] = {}
    param_list[current_header][var_name] = value
    commt_list[current_header][var_name] = comment
        
for line in lines:
    
    # save header only to `param_list`
    header_match = header_pattern.match(line)
    if header_match:
        current_header = header_match.group(1).strip()
        if current_header not in param_list:  # keep the header even if there is no parameter existed
            param_list[current_header] = {}
            commt_list[current_header] = {}
        continue

    # save comments only to `current_comment`
    if line.strip().startswith('!'):
        if current_var:
            current_comment.append(line.strip())
    

In [55]:
header_pattern = re.compile(r"^! === (.+) ===")
block_list = ['KPP%','%KPP','CVMix_CONVECTION%','%CVMix_CONVECTION','CVMIX_DDIFF%','%CVMIX_DDIFF']
block_pattern  = re.compile(r"KPP%|%KPP|CVMix_CONVECTION%|%CVMix_CONVECTION|CVMIX_DDIFF%|%CVMIX_DDIFF")

param_list = {}
commt_list = {}

current_header  = None
current_var     = None
current_value   = []
current_comment = []

def _save_var_to_param_list(var_name,value,comment):
    #if current_header:
    # save parameters and associated comments under each header
    if current_header not in param_list:
        param_list[current_header] = {}
        commt_list[current_header] = {}
    param_list[current_header][var_name] = value
    commt_list[current_header][var_name] = comment
    # else:
    #     param_list[var_name] = value
    #     commt_list[var_name] = comment    

for line in lines:
    # check if line is a header
    header_match = header_pattern.match(line)
    if header_match:
        # save var
        if current_var:
            var_name = current_var
            value    = ''.join(current_value).strip()
            comment  = '\n'.join(current_comment).strip()
            #print(var_name, value, comment)
            _save_var_to_param_list(var_name,value,comment)
        current_header = header_match.group(1).strip()
        #print(param_list)
        current_var = None
        current_value = []
        current_comment = []

        if current_header not in param_list:  # keep the header even if there is no parameter existed
            param_list[current_header] = {}
            commt_list[current_header] = {}
        continue
    # check comment line
    if line.strip().startswith('!'):
        if current_var:
            current_comment.append(line.strip())
        continue

    # check param,value
    if '=' in line:
        # save var
        if current_var:
            var_name = current_var
            value    = ''.join(current_value).strip()
            comment  = '\n'.join(current_comment).strip()
            _save_var_to_param_list(var_name,value,comment)
        param,value = line.split('=',1)
        param = param.strip()
        # separate value and inline comment
        tmp_value = value.split('!')[0].strip() # value
        tmp_commt = value.split('!')[1].strip() if '!' in value else '' # inline comment
        # initialise var
        current_var = param
        current_value = [tmp_value]
        current_comment = [tmp_commt]

    elif block_pattern.search(line): # block_pattern
        if current_var:
            _save_var_to_param_list(current_var, ''.join(current_value).strip(), '\n'.join(current_comment).strip())
        current_var = line.strip()
        current_value = ['']
        current_comment = ['']

# last var
if current_var:
    var_name = current_var
    value    = ''.join(current_value).strip()
    comment  = '\n'.join(current_comment).strip()
    _save_var_to_param_list(var_name,value,comment)

# commt_list
# param_list

def writefile_MOM_input(path_MOM_input,param_list,commt_list,total_width = 32):
    with open(path_MOM_input,'w') as f:
        f.write(f"! This file was written by the script and records the non-default parameters used at run-time.\n")
        for header,variables in param_list.items():
            f.write(f"! === {header} ===\n")
            for var,value in variables.items():
                comment = commt_list[header].get(var,'')
                if comment:
                    comment_lines = comment.split('\n')
                    param_str = f"{var} = {value}"
                    f.write(f"{param_str:<{total_width}} ! {comment_lines[0].strip()}\n")
                    for comment_line in comment_lines[1:]:
                        f.write(f"{'':<{total_width}} {comment_line.strip()}\n")
                elif var in block_list:
                    f.write(f"{var}\n")
                else:
                    f.write(f"{var} = {value}\n")
            f.write("\n")

path_MOM_input = '/g/data/tm70/ml0072/MOM6-CICE6/PROFILING/3_2using_scripts/clx-run-o1344i96c96d48-fcpl-s-fm-F-fr-c_no_diags/MOM_input_writefile4'
writefile_MOM_input(path_MOM_input,param_list,commt_list)

In [95]:
param_list

{'module MOM': {'DIABATIC_FIRST': 'True',
  'USE_REGRIDDING': 'True',
  'THICKNESSDIFFUSE': 'True',
  'THICKNESSDIFFUSE_FIRST': 'True',
  'DT': '1350.0',
  'DT_THERM': '1350.0',
  'HFREEZE': '10.0',
  'DTBT_RESET_PERIOD': '0.0',
  'FRAZIL': 'True',
  'BOUND_SALINITY': 'True',
  'C_P': '3992.0',
  'CHECK_BAD_SURFACE_VALS': 'True',
  'BAD_VAL_SSH_MAX': '50.0',
  'BAD_VAL_SSS_MAX': '75.0',
  'BAD_VAL_SST_MAX': '55.0',
  'BAD_VAL_SST_MIN': '-3.0',
  'SAVE_INITIAL_CONDS': 'True'},
 'module MOM_domains': {'TRIPOLAR_N': 'True',
  'NIGLOBAL': '1440',
  'NJGLOBAL': '1080'},
 'module MOM_fixed_initialization': {'INPUTDIR': '"./input/"'},
 'module MOM_grid_init': {'GRID_CONFIG': '"mosaic"',
  'GRID_FILE': '"ocean_hgrid.nc"',
  'TOPO_CONFIG': '"file"',
  'MINIMUM_DEPTH': '0.5',
  'MAXIMUM_DEPTH': '6000.0',
  'CHANNEL_CONFIG': 'none'},
 'module MOM_verticalGrid': {'NK': '50'},
 'module MOM_EOS': {'DTFREEZE_DP': '-7.75E-08'},
 'module MOM_tracer_flow_control': {'USE_IDEAL_AGE_TRACER': 'True'},
 'mod

In [6]:
namelist = {}
comments = {}

current_var = None

block_pattern = re.compile(r"KPP%|%KPP|CVMix_CONVECTION%|%CVMix_CONVECTION|CVMIX_DDIFF%|%CVMIX_DDIFF")
for line in lines:
    # Check if line is a comment
    if line.strip().startswith('!'):
        if current_var:
            comments[current_var] = line.strip().lstrip('!')
        continue
        
    # Split line into parameter and value
    if '=' in line:
        param, value = line.split('=',1)
        param = param.strip()
        value = value.split('!')[0].strip()  # Remove inline comments
        namelist[param] = value
        current_var = param  # comments
    elif block_pattern.search(line):
        param = line.strip()
        namelist[param] = '' # no comments for block pattern
    
def generate_namelist(namelist,module_name):
    namelist_str = f'&{module_name}\n'
    for param, value in namelist.items():
        comment = comments.get(param, '')
        if comment:
            namelist_str += f"    {param} = {value} ! {comment}\n"
        else:
            namelist_str += f"    {param} = {value}\n"
    return namelist_str

module_name = "MOM"
formatted_namelist = generate_namelist(namelist, module_name)
output_file_path = '/g/data/tm70/ml0072/MOM6-CICE6/COSIMA_MOM6-CICE6/MOM_namelist.nml'
with open(output_file_path, 'w') as f:
    f.write(formatted_namelist)

In [52]:
block_pattern = re.compile(r"KPP%|%KPP|CVMix_CONVECTION%|%CVMix_CONVECTION|CVMIX_DDIFF%|%CVMIX_DDIFF")
remove_pattern = re.compile(r"^! ===")
# Dictionary to store the namelist variables and their values and comments
namelist = {}
comments = {}

current_var = None
current_value = []
current_comment = []

def add_variable_to_namelist(var_name, value, comment):
    if var_name in namelist:
        # If variable already exists, append new value and comment
        namelist[var_name] += ' ' + value
        comments[var_name] += '\n' + comment
    else:
        namelist[var_name] = value
        comments[var_name] = comment

for line in lines:
    # Skip lines matching the exclude pattern
    if exclude_pattern.search(line):
        continue
        
    # Remove lines starting with '==='
    if remove_pattern.match(line):
        continue
        
    # Check if line is a comment
    if line.strip().startswith('!'):
        if current_var:
            current_comment.append(line.strip().lstrip('!'))
        continue
    
    # Check if line contains a variable assignment
    if '=' in line:
        # Save previous variable if it exists
        if current_var:
            add_variable_to_namelist(current_var, ''.join(current_value).strip(), '\n'.join(current_comment).strip())
        
        param, value = line.split('=', 1)
        param = param.strip()
        value = value.split('!')[0].strip()  # Remove inline comments
        
        # Initialize new variable
        current_var = param
        current_value = [value]
        current_comment = []
    elif block_pattern.search(line):
        param = line.strip()
        namelist[param] = '' # no comments for block pattern
# Add the last variable after the loop
if current_var:
    add_variable_to_namelist(current_var, ''.join(current_value).strip(), '\n'.join(current_comment).strip())

# Format the data as a Fortran namelist
def generate_namelist(namelist, comments, module_name):
    namelist_str = f"&{module_name}\n"
    for param, value in namelist.items():
        comment = comments.get(param, '')
        if comment:
            namelist_str += f"    {param} = {value} ! {comment}\n"
        else:
            namelist_str += f"    {param} = {value}\n"
    namelist_str += "/\n"
    return namelist_str

module_name = "MOM"
formatted_namelist = generate_namelist(namelist, comments, module_name)
output_file_path = '/g/data/tm70/ml0072/MOM6-CICE6/COSIMA_MOM6-CICE6/MOM_namelist2.nml'
with open(output_file_path, 'w') as f:
    f.write(formatted_namelist)

In [57]:
# Define patterns to exclude lines
block_pattern = re.compile(r"KPP%|%KPP|CVMix_CONVECTION%|%CVMix_CONVECTION|CVMIX_DDIFF%|%CVMIX_DDIFF")

# Define a pattern to remove lines starting with '==='
remove_pattern = re.compile(r"^! ===")

# Dictionary to store the namelist variables and their values and comments
namelist = {}
comments = {}

current_var = None
current_value = []
current_comment = []

def add_variable_to_namelist(var_name, value, comment):
    if var_name in namelist:
        # If variable already exists, append new value and comment
        namelist[var_name] += ' ' + value
        comments[var_name] += '\n' + comment
    else:
        namelist[var_name] = value
        comments[var_name] = comment


for line in lines:
    
    # Remove lines starting with '==='
    if remove_pattern.match(line):
        continue

    # Check if line is a comment
    if line.strip().startswith('!'):
        if current_var:
            current_comment.append(line.strip().lstrip('!'))
        continue
    
    # Check if line contains a variable assignment
    if '=' in line:
        # Save previous variable if it exists
        if current_var:
            add_variable_to_namelist(current_var, ''.join(current_value).strip(), '\n'.join(current_comment).strip())
        
        param, value = line.split('=', 1)
        param = param.strip()
        # Separate value and inline comment
        value_part = value.split('!')[0].strip()  # Value of the parameter
        comment_part = value.split('!')[1].strip() if '!' in value else ''  # Inline comment
        
        # Initialize new variable
        current_var = param
        current_value = [value_part]
        current_comment = [comment_part]
        
    elif block_pattern.search(line):
        param = line.strip()
        #param = param.replace('%','^')
        namelist[param] = "''" # no comments for block pattern
# Add the last variable after the loop
if current_var:
    add_variable_to_namelist(current_var, ''.join(current_value).strip(), '\n'.join(current_comment).strip())

# Format the data as a Fortran namelist
def generate_namelist(namelist, comments, module_name):
    namelist_str = f"&{module_name}\n"
    for param, value in namelist.items():
        comment = comments.get(param, '')
        # if comment:
        #     namelist_str += f"    {param} = {value} ! {comment}\n"
        # else:
        #     namelist_str += f"    {param} = {value}\n"
        namelist_str += f"    {param} = {value}\n"
    namelist_str += "/\n"
    return namelist_str
    

module_name = "MOM_nml"
formatted_namelist = generate_namelist(namelist, comments, module_name)
output_file_path = '/g/data/tm70/ml0072/MOM6-CICE6/PROFILING/3_2using_scripts/clx-run-o1344i96c96d48-fcpl-s-fm-F-fr-c_no_diags/MOM_namelist4.nml'
with open(output_file_path, 'w') as f:
    f.write(formatted_namelist)

In [89]:
block_list = ['KPP%','%KPP','CVMix_CONVECTION%','%CVMix_CONVECTION','CVMIX_DDIFF%','%CVMIX_DDIFF']
block_list

['KPP%',
 '%KPP',
 'CVMix_CONVECTION%',
 '%CVMix_CONVECTION',
 'CVMIX_DDIFF%',
 '%CVMIX_DDIFF']

In [76]:
commt_list

{'module MOM': {'DIABATIC_FIRST': '[Boolean] default = False\n! If true, apply diabatic and thermodynamic processes, including buoyancy\n! forcing and mass gain or loss, before stepping the dynamics forward.',
  'USE_REGRIDDING': '[Boolean] default = False\n! If True, use the ALE algorithm (regridding/remapping). If False, use the\n! layered isopycnal algorithm.',
  'THICKNESSDIFFUSE': '[Boolean] default = False\n! If true, isopycnal surfaces are diffused with a Laplacian coefficient of KHTH.',
  'THICKNESSDIFFUSE_FIRST': '[Boolean] default = False\n! If true, do thickness diffusion or interface height smoothing before dynamics.\n! This is only used if THICKNESSDIFFUSE or APPLY_INTERFACE_FILTER is true.',
  'DT': '[s]\n! The (baroclinic) dynamics time step.  The time-step that is actually used will\n! be an integer fraction of the forcing time-step (DT_FORCING in ocean-only mode\n! or the coupling timestep in coupled mode.)',
  'DT_THERM': '[s] default = 1800.0\n! The thermodynamic and

In [65]:
# Define patterns to match headers and block parameters
header_pattern = re.compile(r"^! === (.+) ===")
block_pattern = re.compile(r"KPP%|%KPP|CVMix_CONVECTION%|%CVMix_CONVECTION|CVMIX_DDIFF%|%CVMIX_DDIFF")

# Dictionaries to store the namelist variables, their values, and comments
namelist = {}
comments = {}

current_header = None
current_var = None
current_value = []
current_comment = []


def add_variable_to_namelist(var_name, value, comment):
    if current_header:
        if current_header not in namelist:
            namelist[current_header] = {}
            comments[current_header] = {}
        namelist[current_header][var_name] = value
        comments[current_header][var_name] = comment
    else:
        namelist[var_name] = value
        comments[var_name] = comment

for line in lines:
    # Check if line is a header
    header_match = header_pattern.match(line)
    if header_match:
        # Save previous variable if it exists
        if current_var:
            add_variable_to_namelist(current_var, ''.join(current_value).strip(), '\n'.join(current_comment).strip())
        current_header = header_match.group(1).strip()
        current_var = None
        current_value = []
        current_comment = []
        continue

    # Check if line is a comment
    if line.strip().startswith('!'):
        if current_var:
            current_comment.append(line.strip())
        continue
    
    # Check if line contains a variable assignment
    if '=' in line:
        # Save previous variable if it exists
        if current_var:
            add_variable_to_namelist(current_var, ''.join(current_value).strip(), '\n'.join(current_comment).strip())
        
        param, value = line.split('=', 1)
        param = param.strip()
        # Separate value and inline comment
        value_part = value.split('!')[0].strip()  # Value of the parameter
        comment_part = value.split('!')[1].strip() if '!' in value else ''  # Inline comment
        
        # Initialize new variable
        current_var = param
        current_value = [value_part]
        current_comment = [comment_part]
    elif block_pattern.search(line):
        current_var = line.strip()
        current_value = ''
        current_comment = '' 
# Add the last variable after the loop
if current_var:
    add_variable_to_namelist(current_var, ''.join(current_value).strip(), '\n'.join(current_comment).strip())

# # Display the dictionaries
# print("Namelist Variables and Values:")
# for header, vars in namelist.items():
#     print(f"Header: {header}")
#     for var, value in vars.items():
#         print(f"  {var} = {value}")

# print("\nNamelist Variables and Comments:")
# for header, vars in comments.items():
#     print(f"Header: {header}")
#     for var, comment in vars.items():
#         print(f"  {var} : {comment}")

In [54]:
parser = f90nml.Parser()
nml = parser.read('/g/data/tm70/ml0072/MOM6-CICE6/PROFILING/3_2using_scripts/clx-run-o1344i96c96d48-fcpl-s-fm-F-fr-c_no_diags/MOM_namelist4.nml')
#nml['mom_nml']

In [21]:
from collections import OrderedDict

def read_namelist(file_path):
    parameters = OrderedDict()
    comments = OrderedDict()
    current_param = None

    with open(file_path, 'r') as file:
        for line in file:
            # Check for parameter line
            param_match = re.match(r"(\w+)\s*=\s*(\S+)\s*(!.*)?", line)
            if param_match:
                param, value, comment = param_match.groups()
                parameters[param] = value
                comments[param] = [comment] if comment else []
                current_param = param
            elif current_param and line.strip().startswith('!'):
                # Continuation of comments for the last parameter
                comments[current_param].append(line.strip())
            else:
                current_param = None
    
    return parameters, comments

def write_namelist(parameters, comments, file_path):
    with open(file_path, 'w') as file:
        for param, value in parameters.items():
            comment_line = comments[param][0] if comments[param] else ''
            file.write(f"{param} = {value:<20} {comment_line}\n")
            for extra_comment in comments[param][1:]:
                file.write(f"{'':<25} {extra_comment}\n")

# Example usage
input_file = '/g/data/tm70/ml0072/MOM6-CICE6/PROFILING/3_2using_scripts/clx-run-o1344i96c96d48-fcpl-s-fm-F-fr-c_no_diags/MOM_namelist4.nml'  # Path to your input namelist file
output_file = '/g/data/tm70/ml0072/MOM6-CICE6/PROFILING/3_2using_scripts/clx-run-o1344i96c96d48-fcpl-s-fm-F-fr-c_no_diags/output_MOM_input_test'  # Path to the output file

# Read the namelist
parameters, comments = read_namelist(input_file)

# Optionally modify the parameters
#parameters['JOBNAME'] = 'test'

# Write back to the file
write_namelist(parameters, comments, output_file)

In [13]:
startfrom = str(indata['startfrom']).strip().lower().zfill(3)
startfrom

'rest'

In [26]:
def read_yaml(exp_path,yaml_file):
    configpath = Path(exp_path,yaml_file)
    with open(configpath,'r') as f:
        config = yaml.load(f)
    return config,configpath

def write_config_yaml(config,configpath):
    if isinstance(config, CommentedMap):
        # Update specific parameters
        config['jobname'] = exp_name
        with open(configpath,'w') as f:
            yaml.dump(config,f)
                        

In [28]:
ensemble = []
branch = 'master'
branch_new = 'ctrl'

for fname, nmls in indata['namelists'].items():
    print(f'namelists:{fname,nmls}')
    for group, names in nmls.items():
        print(f"nmls: {group,names}")
        for name,values in names.items():
            print(f"names: {name,values}")
            for v in values:
                print(f'value: {v}')
                exp_path = os.path.join(dir_ensemble,'_'.join([template,name,str(v)]))
                print(exp_path)
                rel_path = '_'.join([template,name,str(v)])
                print(rel_path)
                exp_name = os.path.basename(rel_path)
                print(exp_name)

                # if os.path.exists(exp_path):
                #     ensemble.append(exp_path)
                #     continue

                # check if paramter change differs from the template
                
                parser = f90nml.Parser()
                nml = parser.read(Path(templatepath,fname))
                #print(nml)
                print(nml[group][name])
                if v == nml[group][name]:
                    print(' -- not creating ', rel_path, ' - parameters are identical to the template ',template,'\n')
                    continue
                else:
                    print(' -- creating new exp ', rel_path,'\n')

                if os.path.exists(exp_path):
                    print(' -- not creating ', rel_path, ' - already exists!','\n')
                    
                else:
                    print(f'clone template - payu clone!','\n')
                    # clone template payu clone
                    # payu clone -B master -b ctrl test/1deg_jra55_ryf test/1deg_jra55_ryf_test
                    command = f'payu clone -B {branch} -b {branch_new} {templatepath} {exp_path}'
                    test = subprocess.run(command, shell=True, check=False, capture_output=True, text=True)
                    
                nml_tmp = parser.read(Path(exp_path,fname))
                nml_tmp[group][name] = v
                print(f'changed value: {nml_tmp[group][name]}')
                
                with open(Path(exp_path,fname),'w') as nml_file:
                    f90nml.write(nml_tmp,nml_file)
                # config.yaml    
                yaml_file = 'config.yaml'
                config,configpath = read_yaml(exp_path,yaml_file)
                write_config_yaml(config,configpath)
                # MOM_input
                
                
#ensemble

namelists:('ice/cice_in.nml', {'shortwave_nml': {'albicei': [0.3, 0.44, 0.5]}})
nmls: ('shortwave_nml', {'albicei': [0.3, 0.44, 0.5]})
names: ('albicei', [0.3, 0.44, 0.5])
value: 0.3
/g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf_albicei_0.3
test/1deg_jra55_ryf_albicei_0.3
1deg_jra55_ryf_albicei_0.3
0.44
 -- creating new exp  test/1deg_jra55_ryf_albicei_0.3 

clone template - payu clone! 

changed value: 0.3
value: 0.44
/g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf_albicei_0.44
test/1deg_jra55_ryf_albicei_0.44
1deg_jra55_ryf_albicei_0.44
0.44
 -- not creating  test/1deg_jra55_ryf_albicei_0.44  - parameters are identical to the template  test/1deg_jra55_ryf 

value: 0.5
/g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf_albicei_0.5
test/1deg_jra55_ryf_albicei_0.5
1deg_jra55_ryf_albicei_0.5
0.44
 -- creating new exp  test/1deg_jra55_ryf_albicei_0.5 

clone template - payu clone! 

changed value: 0.5


In [46]:
config

{'collate': {'exe': '/g/data/ik11/inputs/access-om2/bin/mppnccombine', 'mem': '30GB', 'ncpus': 4, 'queue': 'normal', 'restart': True, 'walltime': 3600}, 'env': {'UCX_LOG_LEVEL': 'error'}, 'input': '/g/data/ik11/inputs/access-om2/input_20201102/common_1deg_jra55', 'jobname': '1deg_jra55_ryf_albicei_0.5', 'mem': '1000GB', 'model': 'access-om2', 'mpirun': '--mca io ompio --mca io_ompio_num_aggregators 1', 'name': 'common', 'platform': {'nodesize': 48}, 'qsub_flags': '-W umask=027', 'queue': 'normal', 'restart_freq': 1, 'runlog': True, 'stacksize': 'unlimited', 'submodels': [{'exe': '/g/data/ik11/inputs/access-om2/bin/yatm_d750b4b.exe', 'input': ['/g/data/ik11/inputs/access-om2/input_20201102/yatm_1deg', '/g/data/ik11/inputs/JRA-55/RYF/v1-4'], 'model': 'yatm', 'name': 'atmosphere', 'ncpus': 1}, {'exe': '/g/data/ik11/inputs/access-om2/bin/fms_ACCESS-OM_730f0bf_libaccessom2_d750b4b.x', 'input': '/g/data/ik11/inputs/access-om2/input_20201102/mom_1deg', 'model': 'mom', 'name': 'ocean', 'ncpus'

In [17]:
test

CompletedProcess(args='payu clone -B master -b ctrl /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf_albicei_0.5', returncode=0, stdout='Cloned repository from /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf to directory: /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf_albicei_0.5\nCreated and checked out new branch: ctrl\nlaboratory path:  /scratch/tm70/ml0072/access-om2\nbinary path:  /scratch/tm70/ml0072/access-om2/bin\ninput path:  /scratch/tm70/ml0072/access-om2/input\nwork path:  /scratch/tm70/ml0072/access-om2/work\narchive path:  /scratch/tm70/ml0072/access-om2/archive\nUpdated metadata. Experiment UUID: 7787c97c-833f-4ae4-bb3b-f03d2e71327c\nAdded archive symlink to /scratch/tm70/ml0072/access-om2/archive/1deg_jra55_ryf_albicei_0.5-ctrl-7787c97c\nTo change directory to control directory run:\n  cd /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf_albicei_0.5\n', stderr='')

In [18]:
test.returncode

0

In [62]:
os.getcwd()

'/home/563/ml0072'

In [79]:
turningangle = [fname, group, name] == ['ice/cice_in.nml', 'dynamics_nml', 'turning_angle']
turningangle
name

'albicei'

In [ ]:
payu clone -B master -b ctrl 1deg_jra55_ryf_test

In [106]:
!payu --version

payu 1.0.19


In [110]:
test

CompletedProcess(args='!payu clone -B master -b ctrl /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf_albicei_0.5', returncode=127, stdout='', stderr='/bin/sh: !payu: command not found\n')

In [112]:
test

CompletedProcess(args='payu clone -B master -b ctrl /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf /g/data/tm70/ml0072/COMMON/ensemble/test/1deg_jra55_ryf_albicei_0.5', returncode=2, stdout='', stderr="usage: payu [-h] [--version]\n            {archive,build,collate,ghsetup,init,list,profile,push,run,setup,sweep,sync}\n            ...\npayu: error: argument {archive,build,collate,ghsetup,init,list,profile,push,run,setup,sweep,sync}: invalid choice: 'clone' (choose from 'archive', 'build', 'collate', 'ghsetup', 'init', 'list', 'profile', 'push', 'run', 'setup', 'sweep', 'sync')\n")

In [3]:
!payu clone

usage: payu [-h] [--version]
            {archive,build,collate,ghsetup,init,list,profile,push,run,setup,sweep,sync}
            ...
payu: error: argument {archive,build,collate,ghsetup,init,list,profile,push,run,setup,sweep,sync}: invalid choice: 'clone' (choose from 'archive', 'build', 'collate', 'ghsetup', 'init', 'list', 'profile', 'push', 'run', 'setup', 'sweep', 'sync')


In [6]:
!python --version

Python 3.10.14


In [5]:
!python -c 'import payu; print(payu)'

<module 'payu' from '/g/data/hh5/public/apps/miniconda3/envs/analysis3-unstable/lib/python3.10/site-packages/payu/__init__.py'>


In [1]:
import sys
print(sys.executable)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/bin/python


In [2]:
%env PATH=/g/data/vk83/modules/bin:$PATH

env: PATH=/g/data/vk83/modules/bin:$PATH


PATH: /g/data/vk83/apps/payu/1.1.3/bin:/g/data/vk83/modules/bin:/g/data/vk83/modules/bin:$PATH


In [8]:
!payu clone

usage: payu clone [-h] [--model MODEL_TYPE] [--config CONFIG_PATH]
                  [--laboratory LAB_PATH] [-k] [--branch BRANCH]
                  [--new-branch NEW_BRANCH_NAME] [--restart RESTART_PATH]
                  [--parent-experiment PARENT_EXPERIMENT]
                  repository local_directory
payu clone: error: the following arguments are required: repository, local_directory
